In [1]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import muon as mu
import anndata2ri
import logging
import scvi
import os
import scipy
import scipy.io
import scib
import rpy2.rinterface_lib.callbacks

import seaborn as sns
import matplotlib.pyplot as plt

from rpy2.robjects import r
from rpy2.robjects import pandas2ri

rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

import warnings

warnings.filterwarnings("ignore")

ImportError: cannot import name 'logging' from 'scanpy' (unknown location)

In [ ]:
%%R
Sys.setlocale(category = "LC_ALL", locale = "en_US.UTF-8")  # Set the encoding to UTF-8
install.packages("Seurat")
suppressPackageStartupMessages({
    library(Seurat)
})
set.seed(123)

In [ ]:
# adt = sc.read(
#     "/lustre/groups/ml01/workspace/anastasia.litinetskaya/data/neurips-cite/adt_pp.h5ad"
# )
adt = mu.read_10x_mtx(
    'filtered_feature_bc_matrix/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading
adt.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`
adt

In [ ]:
# print("# cells, # genes before filtering:", adt.shape)

# sc.pp.filter_genes(adt, min_counts=3)
# sc.pp.filter_cells(adt, min_counts=3)

# print("# cells, # genes after filtering:", adt.shape)

In [ ]:
n_genes = len(adt["rna"].var_names)
n_regions = len(adt["atac"].var_names)

In [ ]:
adata_paired = sc.read_10x_mtx(
    'filtered_feature_bc_matrix/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True) 
adata_paired.obs["modality"] = "paired"
adata_paired

In [ ]:
adata_mvi = scvi.data.organize_multiome_anndatas(adata_paired)

In [ ]:
scvi.model.MULTIVI.setup_anndata(
    adata_mvi,
    batch_key="modality",
#     categorical_covariate_keys=["batch"],
#     layer="counts",
)

In [ ]:
mvi = scvi.model.MULTIVI(
    adata_mvi,
    n_genes=n_genes,
    n_regions=n_regions,
)

In [ ]:
import os

num_cores = os.cpu_count()
print(num_cores)

In [ ]:
import os
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

In [ ]:
# mvi.train()
mvi.train(devices=num_cores)